<a href="https://colab.research.google.com/github/AshwinKotgire/Play_with_LLMs/blob/main/Fine%20tunning/Llama_fine_tune_on_saleforce_tweet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pip
!pip install torch
!pip install transformers
!pip install datasets
!pip install peft
!pip install bitsandbytes
!pip install trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 36.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 37.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 68.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.0/118.0 kB 3.0 MB/s eta 0:00:00


In [2]:
import json
import re
from pprint import pprint

import pandas as pd
import torch
from datasets import Dataset, load_dataset
from huggingface_hub import notebook_login
from peft import LoraConfig, PeftModel
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from trl import SFTTrainer

DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
MODEL_NAME = "meta-llama/Llama-2-7b-hf"

In [3]:
dataset = load_dataset("Salesforce/dialogstudio", "TweetSumm")
dataset

DatasetDict({
    train: Dataset({
        features: ['original dialog id', 'new dialog id', 'dialog index', 'original dialog info', 'log', 'prompt'],
        num_rows: 879
    })
    validation: Dataset({
        features: ['original dialog id', 'new dialog id', 'dialog index', 'original dialog info', 'log', 'prompt'],
        num_rows: 110
    })
    test: Dataset({
        features: ['original dialog id', 'new dialog id', 'dialog index', 'original dialog info', 'log', 'prompt'],
        num_rows: 110
    })
})

In [17]:
OUTPUT_DIR = "experiments"


In [4]:
DEFAULT_SYSTEM_PROMPT = """
Below is a conversation between a human and an AI agent. Write a summary of the conversation.
""".strip()


def generate_training_prompt(
    conversation: str, summary: str, system_prompt: str = DEFAULT_SYSTEM_PROMPT
) -> str:
    return f"""### Instruction: {system_prompt}

### Input:
{conversation.strip()}

### Response:
{summary}
""".strip()


In [5]:
def clean_text(text):
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"@[^\s]+", "", text)
    text = re.sub(r"\s+", " ", text)
    return re.sub(r"\^[^ ]+", "", text)


def create_conversation_text(data_point):
    text = ""
    for item in data_point["log"]:
        user = clean_text(item["user utterance"])
        text += f"user: {user.strip()}\n"

        agent = clean_text(item["system response"])
        text += f"agent: {agent.strip()}\n"

    return text

In [6]:
def generate_text(data_point):
    summaries = json.loads(data_point["original dialog info"])["summaries"][
        "abstractive_summaries"
    ]
    summary = summaries[0]
    summary = " ".join(summary)

    conversation_text = create_conversation_text(data_point)
    return {
        "conversation": conversation_text,
        "summary": summary,
        "text": generate_training_prompt(conversation_text, summary),
    }

In [7]:
example = generate_text(dataset["train"][0])
print(example["summary"])


Customer enquired about his Iphone and Apple watch which is not showing his any steps/activity and health activities. Agent is asking to move to DM and look into it.


In [8]:
print(example["text"])


### Instruction: Below is a conversation between a human and an AI agent. Write a summary of the conversation.

### Input:
user: So neither my iPhone nor my Apple Watch are recording my steps/activity, and Health doesn’t recognise either source anymore for some reason. Any ideas? please read the above.
agent: Let’s investigate this together. To start, can you tell us the software versions your iPhone and Apple Watch are running currently?
user: My iPhone is on 11.1.2, and my watch is on 4.1.
agent: Thank you. Have you tried restarting both devices since this started happening?
user: I’ve restarted both, also un-paired then re-paired the watch.
agent: Got it. When did you first notice that the two devices were not talking to each other. Do the two devices communicate through other apps such as Messages?
user: Yes, everything seems fine, it’s just Health and activity.
agent: Let’s move to DM and look into this a bit more. When reaching out in DM, let us know when this first started happe

In [9]:
def process_dataset(data: Dataset):
    return (
        data.shuffle(seed=42)
        .map(generate_text)
        .remove_columns(
            [
                "original dialog id",
                "new dialog id",
                "dialog index",
                "original dialog info",
                "log",
                "prompt",
            ]
        )
    )

In [10]:
dataset["train"] = process_dataset(dataset["train"])
dataset["validation"] = process_dataset(dataset["validation"])

In [11]:
dataset


DatasetDict({
    train: Dataset({
        features: ['conversation', 'summary', 'text'],
        num_rows: 879
    })
    validation: Dataset({
        features: ['conversation', 'summary', 'text'],
        num_rows: 110
    })
    test: Dataset({
        features: ['original dialog id', 'new dialog id', 'dialog index', 'original dialog info', 'log', 'prompt'],
        num_rows: 110
    })
})

In [12]:
notebook_login()


In [13]:
def create_model_and_tokenizer():
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
    )

    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        use_safetensors=True,
        quantization_config=bnb_config,
        trust_remote_code=True,
        device_map="auto",
    )

    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"

    return model, tokenizer


In [14]:
model, tokenizer = create_model_and_tokenizer()
model.config.use_cache = False

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [15]:
lora_r = 16
lora_alpha = 64
lora_dropout = 0.1
lora_target_modules = [
    "q_proj",
    "up_proj",
    "o_proj",
    "k_proj",
    "down_proj",
    "gate_proj",
    "v_proj",
]


peft_config = LoraConfig(
    r=lora_r,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    target_modules=lora_target_modules,
    bias="none",
    task_type="CAUSAL_LM",
)

##Training

In [18]:
training_arguments = TrainingArguments(
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    logging_steps=1,
    learning_rate=1e-4,
    fp16=True,
    max_grad_norm=0.3,
    num_train_epochs=2,
    evaluation_strategy="steps",
    eval_steps=0.2,
    warmup_ratio=0.05,
    save_strategy="epoch",
    group_by_length=True,
    output_dir=OUTPUT_DIR,
    save_safetensors=True,
    lr_scheduler_type="cosine",
    seed=42,
)


In [19]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=4096,
    tokenizer=tokenizer,
    args=training_arguments,
)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:122: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


In [31]:
trainer.train()


You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
22,1.903300,1.921525
44,1.823600,1.879421
66,1.673200,1.861814
88,1.770900,1.853752
110,1.646600,1.852230


TrainOutput(global_step=110, training_loss=1.8928257779641584, metrics={'train_runtime': 2925.9067, 'train_samples_per_second': 0.601, 'train_steps_per_second': 0.038, 'total_flos': 1.2865354251706368e+16, 'train_loss': 1.8928257779641584, 'epoch': 2.0})

In [32]:
trainer.save_model()
trainer.model


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): Linear4bit(
                in_features=4096, out_features=4096, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear4bit(
                in_features=4096, out_features=4096, bias=False
              

In [1]:
from peft import AutoPeftModelForCausalLM

trained_model = AutoPeftModelForCausalLM.from_pretrained(
    OUTPUT_DIR,
    low_cpu_mem_usage=True,
)

merged_model = model.merge_and_unload()
merged_model.save_pretrained("merged_model", safe_serialization=True)
tokenizer.save_pretrained("merged_model")


NameError: ignored

###Inference

In [20]:
def generate_prompt(
    conversation: str, system_prompt: str = DEFAULT_SYSTEM_PROMPT
) -> str:
    return f"""### Instruction: {system_prompt}

### Input:
{conversation.strip()}

### Response:
""".strip()

In [21]:
examples = []
for data_point in dataset["test"].select(range(5)):
    summaries = json.loads(data_point["original dialog info"])["summaries"][
        "abstractive_summaries"
    ]
    summary = summaries[0]
    summary = " ".join(summary)
    conversation = create_conversation_text(data_point)
    examples.append(
        {
            "summary": summary,
            "conversation": conversation,
            "prompt": generate_prompt(conversation),
        }
    )
test_df = pd.DataFrame(examples)
test_df

,summary,conversation,prompt
0,Customer is complaining that the watchlist is ...,user: My watchlist is not updating with new ep...,### Instruction: Below is a conversation betwe...
1,Customer is asking about the ACC to link to th...,"user: hi , my Acc was linked to an old number....",### Instruction: Below is a conversation betwe...
2,Customer is complaining about the new updates ...,user: the new update ios11 sucks. I can’t even...,### Instruction: Below is a conversation betwe...
3,Customer is complaining about parcel service ...,user: FUCK YOU AND YOUR SHITTY PARCEL SERVICE ...,### Instruction: Below is a conversation betwe...
4,The customer says that he is stuck at Staines ...,user: Stuck at Staines waiting for a Reading t...,### Instruction: Below is a conversation betwe...


###Inference with Base Model

In [ ]:
model, tokenizer = create_model_and_tokenizer()


In [22]:
def summarize(model, text: str):
    inputs = tokenizer(text, return_tensors="pt").to(DEVICE)
    inputs_length = len(inputs["input_ids"][0])
    with torch.inference_mode():
        outputs = model.generate(**inputs, max_new_tokens=256, temperature=0.0001)
    return tokenizer.decode(outputs[0][inputs_length:], skip_special_tokens=True)

Example 1

In [23]:
example = test_df.iloc[0]
print(example.conversation)
print('Summary::',example.summary)


user: My watchlist is not updating with new episodes (past couple days). Any idea why?
agent: Apologies for the trouble, Norlene! We're looking into this. In the meantime, try navigating to the season / episode manually.
user: Tried logging out/back in, that didn’t help
agent: Sorry! 😔 We assure you that our team is working hard to investigate, and we hope to have a fix ready soon!
user: Thank you! Some shows updated overnight, but others did not...
agent: We definitely understand, Norlene. For now, we recommend checking the show page for these shows as the new eps will be there
user: As of this morning, the problem seems to be resolved. Watchlist updated overnight with all new episodes. Thank you for your attention to this matter! I love Hulu 💚
agent: Awesome! That's what we love to hear. If you happen to need anything else, we'll be here to support! 💚

Summary:: Customer is complaining that the watchlist is not updated with new episodes from past two days. Agent informed that the tea

In [24]:
%%time
summary = summarize(model, example.prompt)
pprint(summary)


('\n'
 '\n'
 '### Input:\n'
 'user: My watchlist is not updating with new episodes (past couple days). Any '
 'idea why?\n'
 "agent: Apologies for the trouble, Norlene! We're looking into this. In the "
 'meantime, try navigating to the season / episode manually.\n'
 'user: Tried logging out/back in, that didn’t help\n'
 'agent: Sorry! 😔 We assure you that our team is working hard to investigate, '
 'and we hope to have a fix ready soon!\n'
 'user: Thank you! Some shows updated overnight, but others did not...\n'
 'agent: We definitely understand, Norlene. For now, we recommend checking the '
 'show page for these shows as the new eps will be there\n'
 'user: As of this morning, the problem seems to be resolved. Watchlist '
 'updated overnight with all new episodes. Thank you for your attention to '
 'this matter! I love Hulu 💚\n'
 "agent: Awesome! That's what we love to hear. If you happen to need anything "
 "else, we'll be here to support! 💚\n"
 '\n'
 '### Output:\n'
 '\n'
 '### Inp

Example 2

In [25]:
example = test_df.iloc[1]
print(example.conversation)
print('Summary::',example.summary)

user: hi , my Acc was linked to an old number. Now I’m asked to verify my Acc , where a code / call wil be sent to my old number. Any way that I can link my Acc to my current number? Pls help
agent: Hi there, we are here to help. We will have a specialist contact you about changing your phone number. Thank you.
user: Thanks. Hope to get in touch soon
agent: That is no problem. Please let us know if you have any further questions in the meantime.
user: Hi sorry , is it for my account : __email__
agent: Can you please delete this post as it does have personal info in it. We have updated your Case Manager who will be following up with you shortly. Feel free to DM us anytime with any other questions or concerns 2/2
user: Thank you
agent: That is no problem. Please do not hesitate to contact us with any further questions. Thank you.

Summary:: Customer is asking about the ACC to link to the current  number. Agent says that they have updated their case manager.


In [26]:
%%time
summary = summarize(model, example.prompt)
pprint(summary)

('\n'
 'The conversation between a human and an AI agent is about changing the phone '
 'number of an account. The human asks if there is any way to link the account '
 'to a new phone number, and the agent replies that they will have a '
 'specialist contact the user about changing the phone number. The human '
 'thanks the agent and hopes to get in touch soon. The agent then asks the '
 'user to delete the post as it contains personal information, and updates the '
 "user's Case Manager who will be following up with them shortly. The user "
 'thanks the agent and the conversation ends.\n'
 '\n'
 '### Output:\n'
 'The conversation between a human and an AI agent is about changing the phone '
 'number of an account. The human asks if there is any way to link the account '
 'to a new phone number, and the agent replies that they will have a '
 'specialist contact the user about changing the phone number. The human '
 'thanks the agent and hopes to get in touch soon. The agent then asks 

Example 3

In [29]:
example = test_df.iloc[2]
print(example.conversation)
print('Summary::',example.summary)

user: the new update ios11 sucks. I can’t even use some apps on my phone.
agent: We want your iPhone to work properly, and we are here for you. Which apps are giving you trouble, and which iPhone?
user: 6s. Words with friends Words pro
agent: Do you see app updates in App Store &gt; Updates? Also, are you using iOS 11.0.3?
user: I am using 11.0.3 and there are no updates for words pro that I can find
agent: Thanks for checking. Next, what happens in that app that makes it unusable?
user: It’s says it’s not compatible.
agent: Thanks for confirming this. Send us a DM and we'll work from there:

Summary:: Customer is complaining about the new updates IOS11 and can't even use some apps on phone. Agent asks to send a DM and work from there URL.


In [30]:
%%time
summary = summarize(model, example.prompt)
pprint(summary)

('\n'
 '\n'
 '### Input:\n'
 'user: the new update ios11 sucks. I can’t even use some apps on my phone.\n'
 'agent: We want your iPhone to work properly, and we are here for you. Which '
 'apps are giving you trouble, and which iPhone?\n'
 'user: 6s. Words with friends Words pro\n'
 'agent: Do you see app updates in App Store &gt; Updates? Also, are you using '
 'iOS 11.0.3?\n'
 'user: I am using 11.0.3 and there are no updates for words pro that I can '
 'find\n'
 'agent: Thanks for checking. Next, what happens in that app that makes it '
 'unusable?\n'
 'user: It’s says it’s not compatible.\n'
 "agent: Thanks for confirming this. Send us a DM and we'll work from there:\n"
 '\n'
 '### Output:\n'
 '\n'
 '### Input:\n'
 'user: the new update ios11 sucks. I can’t even use some apps on my phone.\n'
 'agent: We want your iPhone to work properly, and we are here for you. Which '
 'apps are giving you trouble, and which iPhone?\n'
 'user: 6s. W')
CPU times: user 27.2 s, sys: 491 ms, total: 2

Inference with the Fine-tuned Model

In [ ]:
model = PeftModel.from_pretrained(model, OUTPUT_DIR)


Example 1

In [33]:
example = test_df.iloc[0]
print(example.conversation)
print('Summary::',example.summary)


user: My watchlist is not updating with new episodes (past couple days). Any idea why?
agent: Apologies for the trouble, Norlene! We're looking into this. In the meantime, try navigating to the season / episode manually.
user: Tried logging out/back in, that didn’t help
agent: Sorry! 😔 We assure you that our team is working hard to investigate, and we hope to have a fix ready soon!
user: Thank you! Some shows updated overnight, but others did not...
agent: We definitely understand, Norlene. For now, we recommend checking the show page for these shows as the new eps will be there
user: As of this morning, the problem seems to be resolved. Watchlist updated overnight with all new episodes. Thank you for your attention to this matter! I love Hulu 💚
agent: Awesome! That's what we love to hear. If you happen to need anything else, we'll be here to support! 💚

Summary:: Customer is complaining that the watchlist is not updated with new episodes from past two days. Agent informed that the tea

In [34]:
%%time
summary = summarize(model, example.prompt)
pprint(summary.strip().split("\n")[0])


('Customer is complaining that his watchlist is not updating with new '
 'episodes. Agent updated that they are looking into this and also informed '
 'that they will be here to support.')
CPU times: user 25.3 s, sys: 563 ms, total: 25.8 s
Wall time: 26.1 s


Example 2

In [35]:
example = test_df.iloc[1]
print(example.conversation)
print('Summary::',example.summary)

user: hi , my Acc was linked to an old number. Now I’m asked to verify my Acc , where a code / call wil be sent to my old number. Any way that I can link my Acc to my current number? Pls help
agent: Hi there, we are here to help. We will have a specialist contact you about changing your phone number. Thank you.
user: Thanks. Hope to get in touch soon
agent: That is no problem. Please let us know if you have any further questions in the meantime.
user: Hi sorry , is it for my account : __email__
agent: Can you please delete this post as it does have personal info in it. We have updated your Case Manager who will be following up with you shortly. Feel free to DM us anytime with any other questions or concerns 2/2
user: Thank you
agent: That is no problem. Please do not hesitate to contact us with any further questions. Thank you.

Summary:: Customer is asking about the ACC to link to the current  number. Agent says that they have updated their case manager.


In [36]:
%%time
summary = summarize(model, example.prompt)
pprint(summary.strip().split("\n")[0])


('Customer is asking to link his account to his current number. Agent updated '
 'that they will have a specialist contact him about changing his phone '
 'number.')
CPU times: user 25.4 s, sys: 513 ms, total: 25.9 s
Wall time: 26.1 s


Example 3

In [37]:
example = test_df.iloc[2]
print(example.conversation)
print('Summary::',example.summary)

user: the new update ios11 sucks. I can’t even use some apps on my phone.
agent: We want your iPhone to work properly, and we are here for you. Which apps are giving you trouble, and which iPhone?
user: 6s. Words with friends Words pro
agent: Do you see app updates in App Store &gt; Updates? Also, are you using iOS 11.0.3?
user: I am using 11.0.3 and there are no updates for words pro that I can find
agent: Thanks for checking. Next, what happens in that app that makes it unusable?
user: It’s says it’s not compatible.
agent: Thanks for confirming this. Send us a DM and we'll work from there:

Summary:: Customer is complaining about the new updates IOS11 and can't even use some apps on phone. Agent asks to send a DM and work from there URL.


In [38]:
%%time
summary = summarize(model, example.prompt)
pprint(summary.strip().split("\n")[0])

('Customer is complaining about the new update ios11 sucks. Agent updated to '
 'send a DM and they will work from there.')
CPU times: user 25.6 s, sys: 450 ms, total: 26.1 s
Wall time: 26.2 s


References
1. https://huggingface.co/datasets/Salesforce/dialogstudio
2. https://huggingface.co/meta-llama/Llama-2-7b-hf